# Construcción automatizada de modelos 3D de árboles, a partir de datos LiDAR

**Informe N° 2 RADAR Y LIDAR**

**Estudiante**: Gizela Andrea Guzmán Lugo

**Fecha**: 24/11/2025

El flujo de trabajo aplicado en este proyecto tiene como objetivo la construcción de modelos 3D de árboles a partir de datos LiDAR, con fines de análisis geométrico y semántico. El proceso parte de una nube de puntos clasificada de manera preliminar, para extraer exclusivamente aquellos puntos correspondientes a vegetación. Sobre esta base filtrada, se ejecutan distintas etapas de segmentación, depuración,limpieza y, finalmente, modelado geométrico de árboles.

**Clasificación inicial del LiDAR**: El primer paso consiste en identificar y separar los puntos correspondientes a vegetación dentro de la nube original. Aunque esta clasificación suele ser aproximada, permite aislar ramas, follaje y troncos, eliminando elementos como suelo, edificaciones o vehículos.

**Segmentación individual de árboles**: Una vez aislados los puntos de vegetación, se aplica un proceso de segmentación espacial cuyo propósito es dividir la nube en subconjuntos, donde cada segmento representa idealmente un árbol individual. Esta etapa es fundamental para pasar del nivel de vegetación general al nivel de objeto individual.

**Depuración y limpieza de segmentos**: Después de segmentar los árboles, los datos pueden contener errores de clasificación o puntos atípicos. Se realiza entonces una limpieza por segmentación, filtrando ruido, puntos descolgados o valores extremos que podrían afectar el modelado o el análisis posterior.

**Modelado geométrico de árboles**: Los segmentos limpios se utilizan para reconstruir árboles en diferentes niveles de detalle (LOD). Esto permite representar desde formas simplificadas como cilindros y prismas (LOD1), hasta estructuras más complejas como copas paramétricas o volúmenes ajustados a los datos reales (LOD2 o LOD3).

Para el desarrollo de este trabajo se usan los datos de nubes de puntos disponibles abiertamente (Actueel Hoogtebestand Nederland -AHN3), en la plataforma holandesa de geodatos, Publieke Dienstverlening Op de Kaart (PDOK).

Este trabajo es una implementación de de la línea base presentada en [\"Automatic construction of 3D tree models in
multiple levels of detail from airborne LiDAR data\"](https://repository.tudelft.nl/record/uuid:3e169fc7-5336-4742-ab9b-18c158637cfe), Geert Jan (Rob) de Groot, TU Delft - Architecture and the Built Environment, Master Thesis (2020).

Repositorio Github: [\"TreeConstruction\"](https://github.com/RobbieG91/TreeConstruction/tree/master)


**Importacion de librerias**

In [2]:
import os          # Permite manejar rutas y archivos del sistema operativo
import time        # Mide tiempos de ejecución
import subprocess  # Ejecuta comandos externos del sistema 
import rasterio     # Librería especializada para leer, escribir y manejar datos raster 
import numpy as np # Manejo eficiente de arreglos numéricos, cálculos matemáticos y manipulación de matrices.
import laspy       # Permite leer, modificar y escribir archivos LiDAR (.las y .laz)
import math        # Proporciona funciones matemáticas básicas (trigonometría, potencia, logaritmos)
from sklearn.preprocessing import StandardScaler  # Normaliza y escala valores numéricos 
from sklearn.cluster import DBSCAN               # Algoritmo de clustering 
from sklearn import linear_model                 # Proporciona modelos lineales como RANSAC

**Definición de ruta base y archivo LiDAR de entrada**

In [4]:
# Ruta base donde se almacenan los datos del proyecto
BASE_PATH = r"D:\MODELO_3D\arboles"

# Nombre del archivo LiDAR original (crudo), que contiene los puntos XYZ, intensidad, retornos, etc.
FILENAME  = "Noordereiland.laz"

## 1. Clasificación inicial datos LiDAR

La nube de puntos AHN3 utilizada ya contiene clases predefinidas, pero para este proyecto se trabaja con la clase sin clasificar, con el objetivo de extraer únicamente puntos de vegetación. Para ello, se realiza una reclasificación usando las características espaciales X, Y y la altura normalizada, estimando la rugosidad local de los puntos. Los puntos con alta variabilidad en altura se consideran vegetación, mientras que los de baja variación corresponden a superficies planas (edificios, suelo, vehículos, etc.). Además, se aplica un filtro por altura (>2 m) para eliminar rápidamente objetos cercanos al suelo como coches, mobiliario urbano o ruido.

Para este proceso se utiliza ``LAStools``, una suite especializada para el procesamiento de nubes de puntos LiDAR. La elección de esta herramienta se debe a su alta eficiencia, velocidad de procesamiento y fiabilidad. Como requiere de licencia para el procesamiento de nubes de puntos, se utiiza en modo demo.

**Cambios al código original**

El código fue adaptado para aprovechar las herramientas de LAStools en modo demo, ejecutando directamente los archivos .exe instalados en el computador. Estas aplicaciones fueron descargadas previamente desde la página oficial y se invocan desde el script mediante llamadas al sistema, permitiendo procesar los datos LiDAR sin necesidad de instalar la librería completa en Python.

### 📊 1.1 Normalización alturas

Se creo la función ``normalize_height``, la cual permite normalizar las alturas de un archivo LiDAR (.las o .laz) mediante la herramienta ``lasheight`` de LAStools, con el fin de recalcular la altura de cada punto respecto al terreno (altura relativa) y almacenarla como atributo adicional. Esto es fundamental para tareas de análisis de vegetación de objetos 3D, ya que los modelos basados en altura normalizada permiten identificar los árboles. La función implementa un procesamiento automatizado en Python, verificando la disponibilidad de los archivos, construyendo dinámicamente la ruta de entrada y salida, y ejecutando el comando desde el sistema mediante subprocess.run. Finalmente, genera un nuevo archivo con sufijo _height_veg.laz, que incluye la altura normalizada como atributo, listo para etapas posteriores de segmentación y modelado 3D.

In [34]:
def normalize_height(path, filename_laz):
    """
    Normaliza alturas con lasheight64 (LAStools).
    - path: carpeta base donde están los datos.
    - filename_laz: nombre del archivo .las/.laz original,
    """
    # Ruta completa del archivo de entrada
    in_file = os.path.join(path, filename_laz)
    base_name, ext = os.path.splitext(filename_laz)
    out_file = os.path.join(path, base_name + "_height_veg.laz")

    # Ruta al ejecutable 
    exe = r"D:\MODELO_3D\arboles\LAStools\bin\lasheight64.exe"

    # Comando de ejecucción como lista, con -demo
    cmd = [
        exe,   # Ejecutable de lasheight64
        "-demo",# Modo Demo
        "-i", in_file,
        "-o", out_file,
        "-store_precise_as_extra_bytes", # Guarda alturas normalizadas como extra bytes (mayor precisión)
    ]
    # Muestra el comando antes de ejecutar
    print("COMANDO:", cmd)

    # Ejecutar y ver mensajes
    result = subprocess.run(cmd, capture_output=True, text=True)
    
    # Mostrar resultados de ejecución
    print("RETURN CODE:", result.returncode)
    print("STDOUT:\n", result.stdout)
    print("STDERR:\n", result.stderr)
    
    # Devuelve la ruta del nuevo archivo
    return out_file
    

In [36]:
# 1) Normalizar alturas
normalized = normalize_height(BASE_PATH, FILENAME)
print("NORMALIZED:", normalized, "existe?", os.path.exists(normalized))

COMANDO: ['D:\\MODELO_3D\\arboles\\LAStools\\bin\\lasheight64.exe', '-demo', '-i', 'D:\\MODELO_3D\\arboles\\Noordereiland.laz', '-o', 'D:\\MODELO_3D\\arboles\\Noordereiland_height_veg.laz', '-store_precise_as_extra_bytes']
RETURN CODE: 1
STDOUT:
 
STDERR:
 Please note that LAStools is not "free" (see https://rapidlasso.de/license)
contact 'info@rapidlasso.de' to clarify licensing terms if needed.
         tiny xyz noise. points permuted. intensity, gps_time & point_source_ID zeroed.
done with 'D:\MODELO_3D\arboles\Noordereiland_height_veg.laz'. total time 8.417 sec.

NORMALIZED: D:\MODELO_3D\arboles\Noordereiland_height_veg.laz existe? True


### 🌿 1.2 Clasificación puntos

En esta etapa se emplea `lasclassify64` (LAStools) para reclasificar los puntos del archivo normalizado (*_height_veg.laz*).  

La herramienta utiliza la altura normalizada y medidas de rugosidad local para separar vegetación de superficies planas como suelo y edificaciones. 

Se definen umbrales para:
- `-ground_offset 2.0`: filtrar objetos cercanos al terreno (≤ 2 m).
- `-planar 0.1`: identificar superficies casi planas (tejados, pavimento).
- `-rugged 0.4`: identificar vecindades rugosas asociadas a copas de árboles.  

El archivo de salida *_classified.laz* contiene los puntos ya clasificados.


In [30]:
def classify_points(path, normalized_file):
    """
    Clasifica puntos usando lasclassify64.
    - normalized_file: ruta COMPLETA al archivo *_height_veg.laz devuelto por normalize_height
    """
    # El archivo de entrada es el mismo que devuelve normalize_height
    in_file = normalized_file
    
    # salida: mismo path + sufijo _classified.laz
    base_noext, ext = os.path.splitext(in_file)
    out_file = base_noext + "_classified.laz"
    
    # Ruta al ejecutable de lasclassify64
    exe = r"D:\MODELO_3D\arboles\LAStools\bin\lasclassify64.exe"

    print("IN FILE (classify):", in_file, "existe?", os.path.exists(in_file))
    print("EXE (classify):", exe, "existe?", os.path.exists(exe))
    print("OUT FILE (classify):", out_file)

     # Construcción del comando para lasclassify64
    cmd = [
        exe, # Ejecutable de lasclassify64
        "-demo",                   
        "-i", in_file,
        "-o", out_file,
        "-height_in_attribute", "0", #Indica que la altura normalizada está almacenada en el atributo extra 0
        "-ground_offset", "2.0", # Desplazamiento del terreno: puntos por debajo de 2 m se consideran cercanos al suelo
        "-planar", "0.1", # Umbral para identificar superficies planas (baja variación vertical)
        "-rugged", "0.4", # Umbral para identificar vecindades rugosas (alta variación vertical, típico de vegetación)
    ]
    # Mostrar el comando para verificación
    print("COMANDO classify:", cmd)
    
    # Ejecuta el comando y captura salida y posibles errores
    result = subprocess.run(cmd, capture_output=True, text=True)

    print("RETURN CODE:", result.returncode)
    print("STDOUT:\n", result.stdout)
    print("STDERR:\n", result.stderr)
    
    # Devuelve la ruta del archivo clasificado
    return out_file


In [32]:
# 2) Clasificar
classified = classify_points(BASE_PATH, normalized)
print("CLASSIFIED:", classified, "existe?", os.path.exists(classified))

IN FILE (classify): D:\MODELO_3D\arboles\Noordereiland_height_veg.laz existe? True
EXE (classify): D:\MODELO_3D\arboles\LAStools\bin\lasclassify64.exe existe? True
OUT FILE (classify): D:\MODELO_3D\arboles\Noordereiland_height_veg_classified.laz
COMANDO classify: ['D:\\MODELO_3D\\arboles\\LAStools\\bin\\lasclassify64.exe', '-demo', '-i', 'D:\\MODELO_3D\\arboles\\Noordereiland_height_veg.laz', '-o', 'D:\\MODELO_3D\\arboles\\Noordereiland_height_veg_classified.laz', '-height_in_attribute', '0', '-ground_offset', '2.0', '-planar', '0.1', '-rugged', '0.4']
RETURN CODE: 1
STDOUT:
 
STDERR:
 Please note that LAStools is not "free" (see https://rapidlasso.de/license)
contact 'info@rapidlasso.de' to clarify licensing terms if needed.
and point_source_ID zeroed.
done with 'D:\MODELO_3D\arboles\Noordereiland_height_veg_classified.laz'. total time 30.813 sec.

CLASSIFIED: D:\MODELO_3D\arboles\Noordereiland_height_veg_classified.laz existe? True


### 🌲1.3 Filtrado de vegetación alta

Una vez clasificados los puntos LiDAR, se aplica un filtrado para conservar únicamente la vegetación (clase 5). Para ello, se utiliza `las2las`, una herramienta de LAStools que permite extraer o eliminar clases específicas de un archivo LAS/LAZ.

Aquí filtramos **solo la clase 5 (High Vegetation)**, correspondiente a árboles y arbustos altos, generando el archivo `*_ONLYveg.laz`, que servirá como base para los procesos de segmentación y modelado 3D.


In [34]:
def filter_veg(path, classified_file):
    """
    Filtra solo la vegetación (clase 5) usando las2las64.
    - classified_file: ruta COMPLETA al archivo *_classified.laz
    """
    # Archivo de entrada: resultado del paso anterior (clasificación)
    in_file = classified_file
    # Construcción del nombre del archivo de salida
    base_noext, ext = os.path.splitext(in_file)
    out_file = base_noext + "_ONLYveg.laz"
     
    # Ruta al ejecutable de las2las
    exe = r"D:\MODELO_3D\arboles\LAStools\bin\las2las64.exe"  
    
    # Verificaciones para depuración
    print("IN FILE (filter):", in_file, "existe?", os.path.exists(in_file))
    print("EXE (filter):", exe, "existe?", os.path.exists(exe))
    print("OUT FILE (filter):", out_file)
    
    # Construcción del comando de filtrado
    cmd = [
        exe,
        "-i", in_file,       
        "-o", out_file,
        "-keep_class", "5",   # vegetación alta
    ]
    
    # Mostrar el comando para verificaciones
    print("COMANDO filter:", cmd)
    
    # Ejecutar el comando y capturar mensajes
    result = subprocess.run(cmd, capture_output=True, text=True)

    print("RETURN CODE:", result.returncode)
    print("STDOUT:\n", result.stdout)
    print("STDERR:\n", result.stderr)
    
    # Devolver la ruta del archivo resultante
    return out_file


In [36]:
# 3) Filtrar vegetación
filtered = filter_veg(BASE_PATH, classified)
print("FILTERED:", filtered, "existe?", os.path.exists(filtered))

IN FILE (filter): D:\MODELO_3D\arboles\Noordereiland_height_veg_classified.laz existe? True
EXE (filter): D:\MODELO_3D\arboles\LAStools\bin\las2las64.exe existe? True
OUT FILE (filter): D:\MODELO_3D\arboles\Noordereiland_height_veg_classified_ONLYveg.laz
COMANDO filter: ['D:\\MODELO_3D\\arboles\\LAStools\\bin\\las2las64.exe', '-i', 'D:\\MODELO_3D\\arboles\\Noordereiland_height_veg_classified.laz', '-o', 'D:\\MODELO_3D\\arboles\\Noordereiland_height_veg_classified_ONLYveg.laz', '-keep_class', '5']
RETURN CODE: 0
STDOUT:
 
STDERR:
 
FILTERED: D:\MODELO_3D\arboles\Noordereiland_height_veg_classified_ONLYveg.laz existe? True


## 2. Segmentación individual de árboles

La segmentación de la nube de puntos filtrada (vegetación alta) se realiza mediante un enfoque basado en el algoritmo **Watershed**, implementado en el módulo *Watershed Segmentation* de **System for Automated Geoscientific Analyses (SAGA GIS)**. Este procedimiento permite identificar y separar copas individuales a partir de las variaciones locales de altura, detectando los máximos locales (posibles ápices de los árboles) y delimitando sus zonas de influencia.

El flujo operativo se articula mediante las siguientes etapas:

1️⃣ **Generación del Modelo Digital del Dosel (CHM)**: A partir de la nube LiDAR clasificada y normalizada, se construye un Modelo Digital del Dosel (CHM), calculando la altura máxima de los retornos en cada celda. Este raster representa la estructura vertical del dosel arbóreo y constituye la base para identificar las copas individuales.

2️⃣ **Segmentación de copas mediante Watershed (SAGA GIS)**: Sobre el CHM se ejecuta el algoritmo Watershed, que interpreta el raster como una superficie topográfica invertida. Los máximos locales se reconocen como centros potenciales de copas, mientras que las líneas divisorias entre ellos definen los límites de cada árbol. Como resultado, se obtiene un raster segmentado donde cada píxel contiene un ID único asociado a una copa.

3️⃣ **Transferencia de los segmentos a la nube de puntos**: Finalmente, el raster segmentado se superpone con la nube LiDAR. Mediante operaciones de intersección espacial, a cada punto se le asigna el ID del segmento (copa) correspondiente al píxel donde se ubica. De esta forma, se genera una nube de puntos segmentada árbol por árbol, conservando tanto la estructura 3D como la identidad de cada copa.


### 🟩 2.1 Generación del Modelo Digital del Dosel (CHM)
Dado que el algoritmo de Watershed opera sobre superficies continuas, es necesario convertir previamente la nube de puntos a un raster. A partir de la nube de puntos clasificada como vegetación se crea un raster que representa las alturas máximas del dosel arbóreo. Generando un Modelo Digital del Dosel (CHM), este CHM (Canopy Height Model) captura la estructura vertical de las copas y permite detectar patrones espaciales, como cúspides, bordes y áreas de transición entre árboles. 

Para este propósito se emplea la herramienta `lasgrid` de LAStools, utilizando como atributo de elevación la altura normalizada previamente calculada. Se usa la opción `-highest`, que permite obtener un CHM representando las alturas máximas de la vegetación por celda.  El raster resultante corresponde a un CHM con una resolución espacial de 0.75 metros, el cual ofrece un equilibrio adecuado entre nivel de detalle y eficiencia computacional. 

In [38]:
def create_dem(path, veg_file):
    """
    Crea un raster (DEM/DSM) de 0.75 m usando lasgrid64.
    - veg_file: ruta COMPLETA al archivo *_ONLYveg.laz (o el que quieras rasterizar)
    """
    # Archivo de entrada: nube de puntos filtrada
    in_file = veg_file
    # Definir archivo de salida con sufijo indicativo del DEM
    base_noext, ext = os.path.splitext(in_file)
    out_file = base_noext + "_DEM0_75.tif"
    
    # Ruta al ejecutable de lasgrid64
    exe = r"D:\MODELO_3D\arboles\LAStools\bin\lasgrid64.exe"
    
    # Comprobaciones básicas
    print("IN FILE (grid):", in_file, "existe?", os.path.exists(in_file))
    print("EXE (grid):", exe, "existe?", os.path.exists(exe))
    print("OUT FILE (grid):", out_file)
    
    # Construcción del comando para crear el raster
    cmd = [
        exe,
        "-demo",               
        "-i", in_file,
        "-o", out_file,
        "-attribute", "0",     # altura normalizada en atributo 0
        "-step", "0.75",       # tamaño de celda (m)
        "-highest",            # DSM/canopy (máximo valor en la celda)
    ]

    print("COMANDO grid:", cmd)

    # Ejecutar y capturar salida
    result = subprocess.run(cmd, capture_output=True, text=True)

    # Mostrar resultados
    print("RETURN CODE:", result.returncode)
    print("STDOUT:\n", result.stdout)
    print("STDERR:\n", result.stderr)
    
    # Retornar ruta del archivo generado
    return out_file


In [40]:
# Crear DEM / raster
dem = create_dem(BASE_PATH, filtered)
print("DEM:", dem, "existe?", os.path.exists(dem))

IN FILE (grid): D:\MODELO_3D\arboles\Noordereiland_height_veg_classified_ONLYveg.laz existe? True
EXE (grid): D:\MODELO_3D\arboles\LAStools\bin\lasgrid64.exe existe? True
OUT FILE (grid): D:\MODELO_3D\arboles\Noordereiland_height_veg_classified_ONLYveg_DEM0_75.tif
COMANDO grid: ['D:\\MODELO_3D\\arboles\\LAStools\\bin\\lasgrid64.exe', '-demo', '-i', 'D:\\MODELO_3D\\arboles\\Noordereiland_height_veg_classified_ONLYveg.laz', '-o', 'D:\\MODELO_3D\\arboles\\Noordereiland_height_veg_classified_ONLYveg_DEM0_75.tif', '-attribute', '0', '-step', '0.75', '-highest']
RETURN CODE: 0
STDOUT:
 
STDERR:
 Please note that LAStools is not "free" (see https://rapidlasso.de/license)
contact 'info@rapidlasso.de' to clarify licensing terms if needed.
took 0.073 sec. done with 'D:\MODELO_3D\arboles\Noordereiland_height_veg_classified_ONLYveg_DEM0_75.tif'.

DEM: D:\MODELO_3D\arboles\Noordereiland_height_veg_classified_ONLYveg_DEM0_75.tif existe? True


### 🌐2.2  Segmentación Watershed

Para separar árboles individuales a partir del Modelo Digital del Dosel (CHM), se aplicó el algoritmo Watershed, disponible en el módulo Watershed Segmentation de SAGA GIS. Este método interpreta el CHM como una superficie topográfica, donde las cúspides o máximos locales representan los ápices de las copas, mientras que los límites naturales entre árboles se identifican como líneas divisorias de drenaje.

Mediante el enfoque de cuencas hidrográficas, Watershed detecta y delimita cada copa de forma automática, asignándole un identificador único (ID de segmento) en formato raster.

### 🌀 2.3 Transferencia de los segmentos a la nube de puntos:

Una vez segmentado el Modelo Digital del Dosel (CHM) mediante Watershed en SAGA, cada píxel del raster tiene asignado un ID único que representa una copa o grupo de copas. Ahora es el momento de asignar de nuevo los segmentos creados con el ráster a la nube de puntos. Para ello se transfiere esa información de segmentación desde el raster hacia la nube de puntos, de forma que cada punto LiDAR reciba el ID del árbol al que pertenece.

Para ello se llevan a cabo los siguientes pasos:

1. Se leen los puntos LiDAR (X, Y, Z) y el raster segmentado.
2. Se transforma cada coordenada (X, Y) a la posición (fila, columna) del raster.
3. Se extrae el **SegmentID** correspondiente a ese píxel.
4. Se asigna a cada punto el SegmentID como un nuevo atributo.
5. Se guarda el nuevo archivo LAZ con los IDs por árbol.

El resultado es una **nube LiDAR segmentada por árbol individual**, lista para extracción de métricas y modelado 3D.

**Cambios al código original**

Para transferir esta segmentación de vuelta a la nube de puntos, el autor utilizó el motor de manipulación de características de FME, específicamente mediante el transformador PointCloudOnRaster; mientras que en esta implementación se realizó el mismo procedimiento conceptual mediante una rutina en Python, evitando el uso de herramientas externas.

In [10]:
input_laz  = r"D:\MODELO_3D\arboles\Noordereiland_height_veg_classified_ONLYveg.laz"          # nube de vegetación
seg_raster = r"D:\MODELO_3D\arboles\Noordereiland_height_veg_classified_ONLYveg_DEM0_75_Segments.tif"       # raster con Segment ID (salida de SAGA)
output_laz = r"D:\MODELO_3D\arboles\Noordereiland_segments.laz"

En esta fase, se cargan dos fuentes de datos:

* La nube de puntos LiDAR — contiene coordenadas tridimensionales (X, Y, Z)
* El raster segmentado — generado previamente mediante Watershed, donde cada píxel tiene un ID de segmento (árbol).

In [12]:
# Leer nube LiDAR
las = laspy.read(input_laz)
x = las.x   # coordenadas reales (aplicando escala y offset)
y = las.y

print("Puntos:", len(x))

# Leer raster de segmentos
src = rasterio.open(seg_raster)
seg = src.read(1)              # banda con los IDs
tfm = src.transform            # transformada afín (x,y -> fila,col)
print("Tamaño raster:", seg.shape)

Puntos: 1292642
Tamaño raster: (1048, 1372)


#### Convertir X,Y de los puntos a filas/columnas del raster

Convierte las coordenadas geográficas de cada punto LiDAR (X, Y) a su posición dentro del raster segmentado. Dado que el raster está organizado como una matriz (en filas y columnas), es necesario traducir las coordenadas espaciales a ubicaciones en esta matriz para saber en qué píxel cae cada punto LiDAR.

Una vez obtenidas las filas (rows) y columnas (cols), se crea una máscara lógica (inside) que permite identificar únicamente los puntos que realmente caen dentro del área cubierta por el raster, descartando aquellos que quedan fuera de los límites del segmento.

In [9]:
from rasterio.transform import rowcol

# Obtener fila, columna para cada punto
rows, cols = rowcol(tfm, x, y)

rows = np.array(rows)
cols = np.array(cols)

# Máscara: puntos que caen dentro del raster
h, w = seg.shape
inside = (rows >= 0) & (rows < h) & (cols >= 0) & (cols < w)

print("Puntos dentro del raster:", inside.sum(), "de", len(x))


Puntos dentro del raster: 1292642 de 1292642


#### Crear vector con SegmentID por punto

Se crea un vector llamado segment_id que almacena el identificador del árbol (segmento) para cada punto de la nube LiDAR. Primero, se inicializa el vector con el valor −1, que indica que el punto no pertenece a ningún segmento válido. Luego, solo para los puntos que sí están dentro del raster segmentado, se extrae el valor del píxel correspondiente y se asigna como su SegmentID. Ese número representa el árbol o copa a la que pertenece el punto.

In [ ]:
# Inicializamos con -1 (sin segmento)
segment_id = np.full(len(x), -1, dtype=np.int32)

# Asignar el valor SegmentID del raster a los puntos "inside"
segment_id[inside] = seg[rows[inside], cols[inside]] #solo puntos que caen dentro del raster


#### Guardar segment_id como nuevo atributo en el LAZ

Agrega el vector segment_id como nuevo atributo dentro del archivo LiDAR. Para ello, se usa laspy, que permite manipular archivos LAS/LAZ y manejar atributos adicionales mediante ExtraBytes. Primero se define la dimensión extra, luego se asigna el vector de IDs a cada punto, y finalmente se guarda un nuevo archivo que ya contiene la segmentación por árbol.

In [15]:
from laspy import ExtraBytesParams

# Definir nueva dimensión extra
extra = ExtraBytesParams(name="segment_id", type=np.int32)
las.add_extra_dim(extra)

# Asignar los valores
las["segment_id"] = segment_id

# Escribir nuevo archivo
las.write(output_laz)

print("Archivo guardado en:", output_laz)


Archivo guardado en: D:\MODELO_3D\arboles\Noordereiland_segments.laz


## 3. Depuración y limpieza de segmentos

Antes de extraer parámetros morfométricos por segmento (árbol), se aplica una fase de limpieza para descartar segmentos que no representan árboles reales o que contienen demasiado ruido:

1. **Filtros iniciales**  
   - Mínimo de 50 puntos por segmento.  
   - Intensidad media < 100.  
   - Número medio de retornos > 1.5.  
   - Altura máxima del segmento < 50 m.

2. **Comprobación de planaridad (RANSAC)**  
   - Se ajusta un plano con RANSAC, si la distancia media de los puntos al plano es muy pequeña (segmento casi plano),se descarta porque es probable que sea un tejado, suelo o estructura artificial.

3. **Eliminación de planos parciales y valores atípicos**  
   - Se eliminan subsecciones planas y puntos alejados (outliers) mediante funciones de limpieza (`clean_ransac`, `remove_distant_outliers`), hasta que el segmento contenga únicamente puntos que representen la estructura del árbol.

Solo los segmentos que superan todas estas comprobaciones se consideran **árboles válidos** y pasan a la etapa de cálculo de parámetros (altura, copa, radios, ratios, etc.).

### 🧹3.1 Filtros iniciales y flujo de procesamiento por árbol

Se crea una función llamada `read_trees`, la cual recorre cada segmento (árbol) y filtra aquellos con pocos puntos o con forma plana (posibles techos o ruido). A cada árbol válido se le extraen parámetros estructurales como altura máxima, altura de copa, base de copa, radio superior e inferior, punto de periferia y ratios entre alturas y radios. Finalmente, se almacena un arreglo con los atributos principales de cada árbol, junto con los puntos completos para uso posterior en modelado o clasificación.


In [3]:
def read_trees(path, filename, outfilename):

    readtime = time.perf_counter()
    full_path = os.path.join(path, filename)
    inFile = laspy.read(full_path)
    
    print ("{}{:0.2f}{}{}".format("LAZ reading time: ", time.perf_counter() - readtime, " sec. Number of points in file: ", inFile.__len__()))

    treelist = []
    treelistfull = []

    for seg_id in np.unique(inFile.segment_id):
        for k in range(1):
            segtime = time.perf_counter()
            # saltar el segmento 0 (ruido / sin asignar)
            if seg_id == 0:
                continue
            ""Conversión a una matriz np más pequeña, por lo que ya no es necesario filtrar toda la nube de puntos en los siguientes pasos"."
            rule = inFile.segment_id == seg_id
            seg_array = inFile.points[rule]

            "¿Cuántos puntos tiene realmente un árbol? Esta implementación usa un mínimo de 50 puntos."
            
            # FILTROS INICIALES A NIVEL DE SEGMENTO
            seg_point_count = len(seg_array)
            if seg_point_count < 50:
                continue

            X = seg_array['X']
            Y = seg_array['Y']
            Z = seg_array['Z'].astype(float)  # coordenadas Z originales
            Z_real = seg_array['Z']
            fit_3dfier = int(np.average(Z - Z_real))

            # número mínimo de puntos "internos" que RANSAC debe explicar
            avg_inliers = int(len(seg_array)/100.0*55.0)

            # COMPROBACIÓN DE PLANARIDAD DEL SEGMENTO (RANSAC SOBRE X,Y,Z)
            ransac = linear_model.RANSACRegressor(
                linear_model.LinearRegression(),
                stop_n_inliers=avg_inliers,
                min_samples=3
            )
            ransac.fit(np.array((X, Y)).T, Z)

            inlier_mask = ransac.inlier_mask_
            outlier_mask = np.logical_not(inlier_mask)
            a = np.array((X[inlier_mask], Y[inlier_mask], Z[inlier_mask])).T
            b = np.array((X[inlier_mask], Y[inlier_mask], ransac.predict(np.array((X[inlier_mask], Y[inlier_mask])).T))).T
            # distancia media de los puntos reales al plano ajustado
            avg_distance = np.average(np.sqrt(np.sum((a - b) ** 2, axis=1)))

            """Omitir si el segmento consta de un solo plano"""
            # si la distancia media < 1 m ⇒ segmento casi plano ⇒ no se considera árbol
            if avg_distance < 1:
                continue

            # LIMPIEZA ADICIONAL: PLANOS PARCIALES E INTENSIDAD / RETORNOS
            seg_indexes = np.where(inFile.segment_id == seg_id)[0]
            return_array = inFile.num_returns[seg_indexes]

            return_array, seg_array, X, Y, Z = clean_ransac(return_array, seg_array, X, Y, Z)
            if len(return_array) == 0:
                # This condition is met when either the avg. intensity is too high or the avg. nr of returns is too low.
                continue

            if len(X) == 0:
                continue

            seg_array, X, Y, Z = remove_distant_outliers(seg_array, X, Y, Z)
            if len(X) == 0:
                continue

            if len(seg_array) < 10:
                continue

            """ Nueva función, extracción de parámetros"""

            Z = Z - fit_3dfier

            max_height = np.max(Z)
            min_height = np.min(Z)

            """Cima del árbol: Será la altura máxima o el percentil 99 de altura"""
            tree_top = np.percentile(Z, 99)
            if tree_top/1000.0 > 50:
                continue

            """Base del árbol: Cero, ya que la altura se calcula como la altura desde el suelo. Este valor debe recalcularse 
            posteriormente al valor Z original."""
            tree_base = np.percentile(Z, 1)

            """Base de la copa del árbol: será el 1.º o 5.º percentil de altura."""
            tree_crown_base = np.percentile(Z, 5)

            """Punto Periférico: Será el intervalo de altura donde se ubican la mayoría de los puntos."""
            """Periferia Inferior y Superior: Puntos intermedios entre el Punto Periférico y la Base y la Copa del Árbol, respectivamente."""
            division = np.linspace(min_height, max_height, num=11)
            countlst = []

            for i in range(len(division)-1):
                division_perc = ((i+1)*10)-5.0
                space = np.logical_and(Z >= division[i], Z < division[i+1])

                # Cuando una división no tiene puntos, pasa a la siguiente división.
                div_point_count = np.sum(space)
                if div_point_count == 0:
                    continue

                center = np.average(X[space]), np.average(Y[space])
                coords = np.vstack((X[space], Y[space])).transpose()
                division_height = (division[i]+division[i+1])/2

                radius = np.percentile(np.sqrt(np.sum((coords - center)**2, axis=1)), 99)
                templst = [div_point_count, division_height, division_perc, center, radius, division[i], division[i+1]]
                countlst.append(templst)
            periphery = max(countlst)
            periphery_lower = (periphery[1] + tree_crown_base) / 2.0
            periphery_higher = (periphery[1] + tree_top) / 2.0

            for div in countlst:
                if div[5] <= periphery_lower <= div[6]:
                    radius_lower = div[4]
                    if div[0] < 5:
                        radius_lower = False
                if div[5] <= periphery_higher <= div[6]:
                    radius_higher = div[4]
                    if div[0] < 5:
                        radius_higher = False
            if type(radius_higher) == bool or type(radius_lower) == bool:
                continue

            """Características de los tipos de árboles: relaciones entre las alturas de la periferia, los radios y
            la copa del árbol frente a la base de la copa."""

            height_ratio_hi_lo = periphery_higher / periphery_lower
            height_ratio_hi_per = periphery_higher / periphery[1]
            height_ratio_per_lo = periphery[1] / periphery_lower

            radius_ratio_hi_lo = radius_higher / radius_lower
            radius_ratio_hi_per = radius_higher / periphery[4]
            radius_ratio_per_lo = periphery[4] / radius_lower

            top_base_ratio = tree_top / tree_crown_base
            per_height_per_radius_ratio = periphery[1] / periphery[4]

            tree = [seg_id, periphery[3][0], periphery[3][1], seg_point_count, tree_base, tree_crown_base, periphery[1],
                    periphery[4], periphery_lower, radius_lower, periphery_higher, radius_higher, tree_top,
                    height_ratio_hi_lo, height_ratio_hi_per, height_ratio_per_lo, radius_ratio_hi_lo,
                    radius_ratio_hi_per, radius_ratio_per_lo, top_base_ratio,
                    np.average(seg_array['intensity']), np.average(return_array), per_height_per_radius_ratio]

            treelist.append(tree)
            treelistfull.append(seg_array.array.copy())
            
    if not treelist:
        print("No se encontró ningún árbol válido después de aplicar los filtros.")
        return None  
        
    print("🌳 Total árboles válidos encontrados:", len(treelist))
    
    tree_array = np.vstack(treelist)

    full_tree_array = np.concatenate(treelistfull, axis=0)
    
    outfilename_2 = "{}{}".format(outfilename, "_Full")
    np.save("{}{}{}".format(path, "/Data/Temp/", outfilename), tree_array)
    np.save("{}{}{}".format(path, "/Data/Temp/", outfilename_2), full_tree_array)

    return tree_array
    


### 🔍 3.2 Limpieza de planos y outliers dentro del árbol

La función `clean_ransac`aplica una limpieza más fina a cada segmento (árbol) usando RANSAC:

1. Se seleccionan solo los puntos con un retorno (`return_array == 1`), que suelen corresponder a superficies más compactas (posibles planos).
2. Se descartan segmentos con intensidad media muy alta** (> 180) o número medio de retornos demasiado bajo** (≤ 1.5), ya que probablemente no son árboles.
3. Si hay suficientes puntos con 1 retorno, se ajusta un plano 3D con RANSAC y se calcula la distancia media de los puntos a dicho plano:
   - Si el plano se ajusta muy bien (distancias pequeñas), se considera que esa parte del segmento es un plano (techo, estructura) y se elimina.
   - El proceso puede repetirse recursivamente cambiando los ejes (`swap=True`) para detectar planos en otras orientaciones.
4. Si solo hay unos pocos puntos “sospechosos” con 1 retorno, se eliminan directamente.
5. Si no hay puntos con 1 retorno, la función devuelve el segmento sin cambios.

El resultado son versiones filtradas de `return_array`, `seg_array`, `X`, `Y`, `Z`, con planos y outliers eliminados en la medida de lo posible.


In [5]:
def clean_ransac(return_array, seg_array, X, Y, Z, swap=False):
    """
    Limpia planos y outliers usando RANSAC sobre puntos con 1 retorno.
    Devuelve return_array, seg_array, X, Y, Z filtrados.
    """
    # Índices de puntos con 1 retorno
    clean_indexes = np.where(return_array == 1)[0]
    seg_array_low_nr = seg_array[clean_indexes]

    # Filtros rápidos por intensidad y nº de retornos
    if np.average(seg_array['intensity']) > 180:
        print("high intensity", np.unique(seg_array['segment_id']), np.average(seg_array['intensity']))
        return [], [], [], [], []

    if np.average(return_array) <= 1.5:
        # print("low returns", np.unique(seg_array['segment_id']), np.average(return_array), len(return_array))
        return [], [], [], [], []

    # Caso 1: suficientes puntos con 1 retorno -> intentamos ajustar plano
    if len(seg_array_low_nr) > 10:
        # Tomar coordenadas para RANSAC
        X_P = seg_array_low_nr['X']
        Y_P = seg_array_low_nr['Y']
        Z_P = seg_array_low_nr['height above ground']

        # Si swap=True, cambiamos ejes para intentar mejor ajuste
        if swap:
            Z_P = seg_array_low_nr['Y']
            Y_P = seg_array_low_nr['height above ground']
            Z = seg_array['Y']
            Y = seg_array['height above ground']
            
        # Ajuste de plano mediante RANSAC
        ransac2 = linear_model.RANSACRegressor(
            linear_model.LinearRegression(),
            min_samples=3,
        )
        ransac2.fit(np.array((X_P, Y_P)).T, Z_P)

        inlier_mask2 = ransac2.inlier_mask_
        outlier_mask2 = np.logical_not(inlier_mask2)
        
        # Inliers usados para medir el ajuste del plano
        X_P_I = X_P[inlier_mask2]
        Y_P_I = Y_P[inlier_mask2]
        Z_P_I = Z_P[inlier_mask2]

        inliers = np.array((X_P_I, Y_P_I, Z_P_I)).T
        inlier_predictions = np.array(
            (X_P_I, Y_P_I, ransac2.predict(np.array((X_P_I, Y_P_I)).T))
        ).T
        
        # Distancia media de los inliers al plano → calidad del plano
        avg_distance = np.average(np.sqrt(np.sum((inliers - inlier_predictions) ** 2, axis=1)))

        # Predicciones sobre todos los puntos del árbol
        predictions = np.array((X, Y, ransac2.predict(np.array((X, Y)).T))).T
        allsamples = np.array((X, Y, Z)).T
        
        # Distancia de cada punto al plano
        distances = np.sqrt(np.sum(np.square(allsamples - predictions), axis=1))
        distances2 = []

        for point in allsamples:
            mindist = np.min(np.sqrt(np.sum(np.square(inliers - point), axis=1)))
            distances2.append(mindist)
        distances2 = np.array(distances2)

        # Si el plano ajusta bien (avg_distance pequeña) → eliminar puntos cercanos a ese plano
        if avg_distance < 100:
            deletelist = []

            # Identificar índices de los puntos que forman el plano principal
            for x_p, y_p, z_p in np.nditer((X_P_I, Y_P_I, Z_P_I)):
                xi = np.where(X == x_p)[0]
                yi = np.where(Y == y_p)[0]
                zi = np.where(Z == z_p)[0]
                common_idx = np.intersect1d(xi, np.intersect1d(yi, zi))
                if common_idx.size > 0:
                    deletelist.append(common_idx[0])
                    
            # Más puntos a eliminar: cerca del plano y de los inliers
            stuff = np.where((distances < 750) & (distances2 < 2000))[0]
            mask_stuff = np.logical_not(np.isin(stuff, deletelist))

            deletelist.extend(stuff[mask_stuff])
            
            # Construir máscara inversa (puntos que se quedan)
            mask = ~np.isin(np.arange(len(seg_array)), deletelist)

            seg_array = seg_array[mask]
            return_array = return_array[mask]
            X = X[mask]
            Y = Y[mask]
            Z = Z[mask]
            
            # Si se eliminaron todos los puntos, el segmento deja de ser válido
            if len(seg_array) == 0:
                return [], [], [], [], []
                
            # Si aún hay suficientes puntos, repetir limpieza con swap=True
            if len(seg_array) > 100:   # solo si hay suficientes puntos
                return_array, seg_array, X, Y, Z = clean_ransac(return_array, seg_array, X, Y, Z, swap=True)

            return return_array, seg_array, X, Y, Z

        # Si el plano no es bueno y aún no hemos swapeado ejes, probar de nuevo con swap=True
        if avg_distance >= 100 and swap is False:
            return clean_ransac(return_array, seg_array, X, Y, Z, swap=True)
        else:
            if swap:
                return return_array, seg_array, X, Z, Y
            else:
                return return_array, seg_array, X, Y, Z

    # Caso 2: pocos puntos problemáticos (0 < len < 10) -> borrarlos todos sin buscar matching fino
    if 0 < len(seg_array_low_nr) < 10:
        """
        Si hay muy pocos puntos con estas propiedades cuestionables,
        elimínalos todos directamente, sin intentar planos.
        """
        mask = np.ones(len(seg_array), dtype=bool)
        mask[clean_indexes] = False # quitar todos los puntos con 1 retorno

        seg_array = seg_array[mask]
        return_array = return_array[mask]
        X = X[mask]
        Y = Y[mask]
        Z = Z[mask]

        if swap:
            return return_array, seg_array, X, Z, Y
        else:
            return return_array, seg_array, X, Y, Z

    # Caso 3: no hay puntos low_nr -> no limpiamos nada
    if swap:
        return return_array, seg_array, X, Z, Y
    else:
        return return_array, seg_array, X, Y, Z


### ⚠️3.3 Depuración de valores atípicos

La función `remove_distant_outliers` elimina puntos atípicos (outliers) dentro de cada árbol usando **DBSCAN**:

1. Se reconstruyen las coordenadas X, Y y la altura normalizada (Z = height above ground).
2. Se normalizan las coordenadas y se aplica DBSCAN para detectar:
   - Ruido (`labels == -1`): puntos muy aislados.
   - Clusters secundarios: grupos pequeños alejados del árbol principal.
3. Si se detecta ruido, se prueba iterativamente con distintos valores de `eps`  
   (0.50, 0.75, 1.0, 1.5, 2.0) para encontrar una separación razonable entre árbol real y ruido.
4. Si hay varios clusters, se conserva solo el cluster más grande (que se asume como el árbol) 
   y se eliminan los demás.
5. Finalmente, si la cantidad de puntos eliminados es pequeña (≤ 5 %), se borran del segmento y,
   si aún quedan suficientes puntos (> 50), el proceso se repite recursivamente.

El resultado es un conjunto de puntos por árbol más compacto y coherente, sin puntos “perdidos” o
clusters pequeños que podrían distorsionar las métricas del árbol.


In [9]:
def remove_distant_outliers(seg_array, X, Y, Z):
   
    # Recalcular X, Y, Z desde seg_array para garantizar coherencia
    X = seg_array['X']
    Y = seg_array['Y']
    Z = seg_array['height above ground']  

    # Construir coordenadas
    coords = np.vstack((X, Y, Z)).T
    #Escalar
    coordsnorm = StandardScaler().fit_transform(coords)
    
    # DBSCAN inicial: eps pequeño → vecindades compactas
    db = DBSCAN(eps=0.50, min_samples=50).fit(coordsnorm)
    labels = db.labels_
    nrpoints = len(seg_array)
    remove = []

    """Eliminar ruido"""
    #(labels == -1) ajustando eps si es necesario
    if np.any(labels == -1):
        remove = np.where(labels == -1)[0]
        if len(remove) > (0.05 * nrpoints) or len(remove) == 0:
            db = DBSCAN(eps=0.75, min_samples=50).fit(coordsnorm)
            labels = db.labels_
            if np.any(labels == -1):
                remove = np.where(labels == -1)[0]
                if len(remove) > (0.05 * nrpoints) or len(remove) == 0:
                    db = DBSCAN(eps=1.0, min_samples=50).fit(coordsnorm)
                    labels = db.labels_
                    if np.any(labels == -1):
                        remove = np.where(labels == -1)[0]
                        if len(remove) > (0.05 * nrpoints) or len(remove) == 0:
                            db = DBSCAN(eps=1.5, min_samples=50).fit(coordsnorm)
                            labels = db.labels_
                            if np.any(labels == -1):
                                remove = np.where(labels == -1)[0]
                                if len(remove) > (0.05 * nrpoints) or len(remove) == 0:
                                    db = DBSCAN(eps=2.0, min_samples=50).fit(coordsnorm)
                                    labels = db.labels_
                                    if np.any(labels == -1):
                                        remove = np.where(labels == -1)[0]

    """En caso de varios clústeres -> Mantener solo el clúster más grande."""
    if len(np.unique(labels)) > 2:
        clustercount = []
        for i in np.unique(labels):
            clustercount.append((len(np.where(labels == i)[0]), i))
        keeplabel = max(clustercount)[1]
        remove = np.where(labels != keeplabel)[0]

    #Aplicar eliminación si la proporción de puntos a borrar es razonable 
    if len(remove) > 0 and len(remove) <= 0.05 * nrpoints:
        mask = ~np.isin(np.arange(len(seg_array)), remove)
        seg_array = seg_array[mask]
        X = X[mask]
        Y = Y[mask]
        Z = Z[mask]

        # Recursión SI aún hay outliers y suficientes puntos
        if len(seg_array) > 50:
            return remove_distant_outliers(seg_array, X, Y, Z)
            
  # En cualquier otro caso, devuelve el segmento tal como está
    else:
        return seg_array, X, Y, Z


LAZ reading time: 0.09 sec. Number of points in file: 1292642
🌳 Total árboles válidos encontrados: 685
Total Calculation time: 97.44 sec.


Este bloque define los archivos de entrada y salida, ejecuta la función de procesamiento de árboles (`read_trees`) y mide el tiempo total de cálculo. Al finalizar, muestra por consola cuánto tardó en procesarse toda la nube LiDAR segmentada.

In [7]:
Ruta = r"D:\MODELO_3D\arboles"      
Nombre_entrada = "Noordereiland_segments.laz"
Nombre_salida = "Noordereiland_segments_AllFeatures"

if __name__ == '__main__':
   """Punto de ejecución principal.
   Se define la ruta y los archivos de entrada y salida."""
    
    starttime = time.perf_counter()

    read_trees(Ruta, Nombre_entrada, Nombre_salida)

    elapsed_time = time.perf_counter() - starttime
    print(f"Total Calculation time: {elapsed_time:.2f} sec.")

## 4. Preparación del archivo para modelado y clasificación 

En la metodología original, una vez finalizada la etapa de limpieza y parametrización de los segmentos arbóreos, se ejecuta un procedimiento de clasificación automática del tipo de árbol (hoja caduca, conífera, ornamental, etc.). Esta clasificación se lleva a cabo mediante el entrenamiento de una red neuronal supervisada, cuyos modelos (ML_Classification_NN_Generator.py y ML_TreeType_Classifier.py) permiten predecir la clase tipológica de nuevos árboles detectados en la nube de puntos, con un valor asociado de probabilidad o classification certainty.

Estos atributos (TreeType y Classification_Certainty) son fundamentales en el flujo original, ya que se incorporan posteriormente al archivo CityJSON como metadatos semánticos, y además permiten ajustar la representación geométrica del árbol según su especie (por ejemplo, copa cónica en coníferas o esférica en caducifolios).

Sin embargo, en el presente trabajo no se dispone del modelo previamente entrenado, ni de un conjunto de datos rotulados necesarios para llevar a cabo el entrenamiento supervisado. Por tanto, la etapa de clasificación automática no fue implementada.

Con el objetivo de mantener la compatibilidad con la estructura de datos esperada por el script de reconstrucción geométrica, se optó por simular la salida de dicho clasificador, sin modificar la lógica original del modelo.

Se procedió a extender el archivo Noordereiland_segments_AllFeatures.npy, que contiene los parámetros geométricos de cada árbol, incorporando manualmente dos nuevas columnas:

| Nombre del campo           | Valor asignado  | Propósito                                |
| -------------------------- | --------------- | ---------------------------------------- |
| `TreeType`                 | `"GenericTree"` | Simula la etiqueta de especie predicha   |
| `Classification_Certainty` | `1.0` (o 0.8)   | Representa confianza ficticia del modelo |


In [3]:
# Ruta a la carpeta raíz del repositorio
path = r"D:\MODELO_3D\arboles"   

# Nombre del archivo que sale de Data_Cleaning_Parameter_Extract
input_name = "Noordereiland_segments_AllFeatures.npy"

# Nombre del archivo de salida (formato “_ML” compatible con json_writer)
output_name = "Noordereiland_segments_AllFeatures_ML.npy"

# Ruta completa de entrada y salida (usa la estructura Data/Temp del repo)
in_path = os.path.join(path, "Data", "Temp", input_name)
out_path = os.path.join(path, "Data", "Temp", output_name)

print("Leyendo:", in_path)
tree_array = np.load(in_path)    # (N, 23) aprox

print("Shape original:", tree_array.shape)

N = tree_array.shape[0]

# Columna de tipo de árbol (string)
# Ponemos un tipo genérico para todos
tree_type = np.full((N, 1), "GenericTree", dtype=object)
# Ejemplo alternativa:
# tree_type = np.full((N, 1), "Coniferae", dtype=object)

# Columna de certeza de clasificación (float)
certainty = np.ones((N, 1), dtype=float) * 0.9

# Apilar -> ahora serán 25 columnas (0–22 parámetros, 23 tipo, 24 certeza)
output_array = np.column_stack((tree_array, tree_type, certainty))

print("Shape final (debería ser N x 25):", output_array.shape)

# Guardar
np.save(out_path, output_array)
print("Guardado en:", out_path)


Leyendo: D:\MODELO_3D\arboles\Data\Temp\Noordereiland_segments_AllFeatures.npy
Shape original: (542, 23)
Shape final (debería ser N x 25): (542, 25)
Guardado en: D:\MODELO_3D\arboles\Data\Temp\Noordereiland_segments_AllFeatures_ML.npy


C:\Users\Estacion_1\AppData\Local\Temp\ipykernel_22652\2657706181.py:19: UserWarning: Reading `.npy` or `.npz` file required additional header parsing as it was created on Python 2. Save the file again to speed up loading and avoid this warning.
  tree_array = np.load(in_path)    # (N, 23) aprox


## 5. Modelado Geométrico de árboles

En esta etapa se describen los pasos seguidos para construir los modelos 3D de árboles en los distintos niveles de detalle (LOD). los modelos se generan siguiendo las especificaciones de CityJSON, de modo que cada árbol se almacena como un CityObject, ya que el resultado del proceso son árboles individuales.Todos los modelos implícitos se basan en geometrías hexagonales, una decisión tomada para reducir el número de vértices necesarios en comparación con formas más complejas.

Para ello se crea la función `write_cityJSON`, la cual toma como entrada los parámetros geométricos de cada árbol y genera un archivo **CityJSON** con objetos del tipo `SolitaryVegetationObject`.

En función del nivel de detalle (`lod`), se construyen:

- **LOD0**: un hexágono plano que representa la proyección de la copa.
- **LOD1**: un prisma hexagonal elevado, que modela un volumen simple de copa.
- **LOD2**: un modelo paramétrico compuesto por tronco y varias “coronas” hexagonales que
  describen mejor la forma de la copa.
- **LOD3**: una geometría envolvente basada en la nube de puntos, combinada con un tronco paramétrico.

El resultado se exporta en formato `.json` bajo la estructura CityJSON, incluyendo materiales para el tronco y la copa, y atributos básicos por árbol (ID de segmento, número de puntos, intensidad y número de retornos).


In [ ]:
import time                     # Medir tiempos de ejecución (cálculos, loops, exportación)
import json                     # Crear y guardar archivos en formato CityJSON (.json)
import math                     # Funciones matemáticas (senos, cosenos, radianes, etc. para hexágonos)
import matplotlib.pyplot as plt # Visualización básica (gráficos, nubes de puntos, ver formas de árbol)
from mpl_toolkits.mplot3d import Axes3D   # Visualización 3D de nubes de puntos y modelos
from scipy.spatial import Delaunay        # Generación de triangulación para Alpha-shapes (LOD3)
from scipy.spatial import ConvexHull      # Cálculo del casco convexo (ConvexHull) de copas en LOD3
from collections import defaultdict        # Gestión eficiente de listas de triángulos únicos (Alpha-shapes)
from sklearn.preprocessing import MinMaxScaler  # Normalización de coordenadas (Alpha-shapes y convex hull)
import sys                      # Manejo del sistema (control de rutas, interrupciones, debugging)

In [21]:
def write_cityJSON(path, filename, lod, outfilename, param_filename, convex):
    
    # filename → archivo de puntos (dict o array estructurado)
    full_points_path = os.path.join(path, "Data", "Temp", filename)
    tree_array = np.load("{}{}{}".format(path, "/Data/Temp/", filename), allow_pickle=True)
    
    # Si viene como array de objetos, extraemos el diccionario
    if isinstance(tree_array, np.ndarray) and tree_array.dtype == object:
        tree_array = tree_array.item()

    if param_filename:
        param_array = np.load(f"{path}/Data/Temp/{param_filename}", allow_pickle=True)
    jsondict = {
        "type": "CityJSON",
        "version": "1.0"
    }
    jsondict['CityObjects'] = {}
    jsondict['vertices'] = []

    vcounter = 0

    jsondict['appearance'] = {
        "materials": [
            {
                "name": "TreeTrunk",
                "ambientIntensity": 0.2000,
                "diffuseColor": [0.6, 0.4, 0.1],
                "shininess": 0.2,
                "transparency": 0.0,
                "isSmooth": False
            },
            {
                "name": "GenericTreeCrown",
                "ambientIntensity": 0.2000,
                "diffuseColor": [0.41, 0.61, 0.35],
                "shininess": 0.2,
                "transparency": 0.0,
                "isSmooth": False
            },
            {
                "name": "Yellowish",
                "ambientIntensity": 0.2000,
                "diffuseColor": [0.85, 0.85, 0.56],
                "shininess": 0.2,
                "transparency": 0.0,
                "isSmooth": False
            },
            {
                "name": "MossyGreen",
                "ambientIntensity": 0.2000,
                "diffuseColor": [0.48, 0.54, 0.23],
                "shininess": 0.2,
                "transparency": 0.0,
                "isSmooth": False
            },
            {
                "name": "BlueishGreen",
                "ambientIntensity": 0.2000,
                "diffuseColor": [0.35, 0.40, 0.31],
                "shininess": 0.2,
                "transparency": 0.0,
                "isSmooth": False
            },
            {
                "name": "VomitGreen",
                "ambientIntensity": 0.2000,
                "diffuseColor": [0.58, 0.60, 0.38],
                "shininess": 0.2,
                "transparency": 0.0,
                "isSmooth": False
            },
            {
                "name": "Grayish",
                "ambientIntensity": 0.2000,
                "diffuseColor": [0.72, 0.75, 0.67],
                "shininess": 0.2,
                "transparency": 0.0,
                "isSmooth": False
            }

        ]
    }

    # Escritor para LOD, 0, 1 y 2
    if lod != 3:
        for tree in tree_array:
            jsondict['CityObjects'][tree[0]] = {
                "type": "SolitaryVegetationObject",
            }

            starttime = time.perf_counter()

            """Crear vértices para el modelo de árbol"""
            """Parametros"""

            x = float(tree[1]) / 1000
            y = float(tree[2]) / 1000
            z_b = float(tree[4]) / 1000
            z_c = float(tree[5]) / 1000
            z_p = float(tree[6]) / 1000
            r_p = float(tree[7]) / 1000
            z_pl = float(tree[8]) / 1000
            r_pl = float(tree[9]) / 1000
            z_ph = float(tree[10]) / 1000
            r_ph = float(tree[11]) / 1000
            z_t = float(tree[12]) / 1000
            t = math.radians(60)

            
            colorvalue = 3

            """LOD0: Hexágono parametrizado"""
            if lod == 0:

                """Centro, Altura de la base"""
                v1 = [x, y, z_b]

                """Periferia"""
                v2 = [x - r_p, y, z_b]
                v3 = [x - (math.cos(t) * r_p), y + (math.sin(t) * r_p), z_b]
                v4 = [x + (math.cos(t) * r_p), y + (math.sin(t) * r_p), z_b]
                v5 = [x + r_p, y, z_b]
                v6 = [x + (math.cos(t) * r_p), y - (math.sin(t) * r_p), z_b]
                v7 = [x - (math.cos(t) * r_p), y - (math.sin(t) * r_p), z_b]

                boundaries = []
                values = []

                """Crear índices para el modelo de árbol"""
                """Base del tronco del árbol"""
                for i in range(1, 7):
                    if i != 6:
                        boundaries.append([[vcounter, vcounter + i + 1, vcounter + i]])
                        values.append(colorvalue)
                    else:
                        boundaries.append([[vcounter, vcounter + i - 5, vcounter + i]])
                        values.append(colorvalue)

                for i in range(7):
                    jsondict['vertices'].append(
                        list(np.around(eval("{}{}".format("v", i+1)), decimals=2))
                    )

                              """
                jsondict['CityObjects'][tree[0]]['attributes'] = {
                    "TreeType": tree[23],
                    "Classification_Certainty": tree[24],
                    "Point Count": tree[3],
                    "Height Crown Base": z_c,
                    "Periphery Height": z_p,
                    "Periphery Radius": r_p,
                    "Lower Periphery Height": z_pl,
                    "Lower Periphery Radius": r_pl,
                    "Higher Periphery Height": z_ph,
                    "Higher Periphery Radius": r_ph,
                    "Tree Top": z_t,
                    "Ratio Height High + Low": tree[13],
                    "Ratio Height High + Periphery": tree[14],
                    "Ratio Height Periphery + Low": tree[15],
                    "Ratio Radius High + Low": tree[16],
                    "Ratio Radius High + Periphery": tree[17],
                    "Ratio Radius Periphery + Low": tree[18],
                    "Ratio Height Tree Top + Crown Base": tree[19],
                    "Average Intensity": tree[20],
                    "Average Number of Returns": tree[21],
                    "Ratio Periphery Height + Periphery Radius": tree[22]
                }
                """
                jsondict['CityObjects'][tree[0]]['attributes'] = {
                    "Segment ID": int(tree[0]),
                    "Point Count": int(tree[3]),
                    "Average Intensity": float(tree[20]),
                    "Average Number of Returns": float(tree[21])
                }


                jsondict['CityObjects'][tree[0]]['geometry'] = [{
                    "type": "MultiSurface",
                    "lod": lod,
                    "boundaries": boundaries,
                    "material": {
                        "visual": {
                            "values": values
                        },
                    },
                }]

                vcounter += 7


            """LOD1: Hexágono elevado parametrizado"""
            if lod == 1:
                """Centro, Altura de la base"""
                v1 = [x, y, z_b]

                """Límite inferior"""
                v2 = [x - r_p, y, z_b]
                v3 = [x - (math.cos(t) * r_p), y + (math.sin(t) * r_p), z_b]
                v4 = [x + (math.cos(t) * r_p), y + (math.sin(t) * r_p), z_b]
                v5 = [x + r_p, y, z_b]
                v6 = [x + (math.cos(t) * r_p), y - (math.sin(t) * r_p), z_b]
                v7 = [x - (math.cos(t) * r_p), y - (math.sin(t) * r_p), z_b]

                """Límite superior"""
                v8 = [x - r_p, y, z_t]
                v9 = [x - (math.cos(t) * r_p), y + (math.sin(t) * r_p), z_t]
                v10 = [x + (math.cos(t) * r_p), y + (math.sin(t) * r_p), z_t]
                v11 = [x + r_p, y, z_t]
                v12 = [x + (math.cos(t) * r_p), y - (math.sin(t) * r_p), z_t]
                v13 = [x - (math.cos(t) * r_p), y - (math.sin(t) * r_p), z_t]

                """Copa del árbol"""
                v14 = [x, y, z_t]

                boundaries = []
                values = []

                """Crear índices para el modelo de árbol"""
                """Hexágono elevado en tierra"""
                for i in range(1, 7):
                    if i != 6:
                        boundaries.append([[vcounter, vcounter + i, vcounter + i + 1]])
                        values.append(colorvalue)
                    else:
                        boundaries.append([[vcounter, vcounter + i, vcounter + i - 5]])
                        values.append(colorvalue)

                """Lados hexagonales elevados"""
                for i in range(1, 7):
                    if i != 6:
                        boundaries.append([[vcounter + i, vcounter + i + 6, vcounter + i + 7, vcounter + i + 1]])
                        values.append(colorvalue)
                    else:
                        boundaries.append([[vcounter + i, vcounter + i + 6, vcounter + i + 1, vcounter + i - 5]])
                        values.append(colorvalue)

                """Copa de árbol hexagonal elevada"""
                for i in range(7, 13):
                    if i != 12:
                        boundaries.append([[vcounter + 13, vcounter + i + 1, vcounter + i]])
                        values.append(colorvalue)
                    else:
                        boundaries.append([[vcounter + 13, vcounter + i - 5, vcounter + i]])
                        values.append(colorvalue)

                for i in range(14):
                    jsondict['vertices'].append(
                        list(np.around(eval("{}{}".format("v", i + 1)), decimals=2))
                    )

                """
                jsondict['CityObjects'][tree[0]]['attributes'] = {
                    "TreeType": tree[23],
                    "Classification_Certainty": tree[24],
                    "Point Count": tree[3],
                    "Height Crown Base": z_c,
                    "Periphery Height": z_p,
                    "Periphery Radius": r_p,
                    "Lower Periphery Height": z_pl,
                    "Lower Periphery Radius": r_pl,
                    "Higher Periphery Height": z_ph,
                    "Higher Periphery Radius": r_ph,
                    "Tree Top": z_t,
                    "Ratio Height High + Low": tree[13],
                    "Ratio Height High + Periphery": tree[14],
                    "Ratio Height Periphery + Low": tree[15],
                    "Ratio Radius High + Low": tree[16],
                    "Ratio Radius High + Periphery": tree[17],
                    "Ratio Radius Periphery + Low": tree[18],
                    "Ratio Height Tree Top + Crown Base": tree[19],
                    "Average Intensity": tree[20],
                    "Average Number of Returns": tree[21],
                    "Ratio Periphery Height + Periphery Radius": tree[22]
                }
                """

                jsondict['CityObjects'][tree[0]]['attributes'] = {
                    "Segment ID": int(tree[0]),
                    "Point Count": int(tree[3]),
                    "Average Intensity": float(tree[20]),
                    "Average Number of Returns": float(tree[21])
                }

                jsondict['CityObjects'][tree[0]]['geometry'] = [{
                    "type": "MultiSurface",
                    "lod": lod,
                    "boundaries": boundaries,
                    "material": {
                        "visual": {
                            "values": values
                        },
                    },
                }]

                vcounter += 14

            """LOD2: Modelo de árbol parametrizado"""
            if lod == 2:
                # Ajuste del tronco en LOD2
                height = z_t - z_b
                if height <= 0:
                # si los parámetros vienen mal, lo saltamos
                    continue

                # Tronco llega al 40% de la altura del árbol
                z_c = z_b + 0.55 * height

                # Radio del tronco proporcional a la copa
                # 25% del radio de copa, mínimo 0.15 m (15 cm)
                r_t = max(0.25 * r_p, 0.15)  

                """Centro, Altura de la base"""
                v1 = [x, y, z_b]

                """Tronco, Tierra"""
                v2 = [x - r_t, y, z_b]
                v3 = [x - (math.cos(t) * r_t), y + (math.sin(t) * r_t), z_b]
                v4 = [x + (math.cos(t) * r_t), y + (math.sin(t) * r_t), z_b]
                v5 = [x + r_t, y, z_b]
                v6 = [x + (math.cos(t) * r_t), y - (math.sin(t) * r_t), z_b]
                v7 = [x - (math.cos(t) * r_t), y - (math.sin(t) * r_t), z_b]

                """Base de la corona"""
                v8 = [x - r_t, y, z_c]
                v9 = [x - (math.cos(t) * r_t), y + (math.sin(t) * r_t), z_c]
                v10 = [x + (math.cos(t) * r_t), y + (math.sin(t) * r_t), z_c]
                v11 = [x + r_t, y, z_c]
                v12 = [x + (math.cos(t) * r_t), y - (math.sin(t) * r_t), z_c]
                v13 = [x - (math.cos(t) * r_t), y - (math.sin(t) * r_t), z_c]

                """Límite inferior de la periferia"""
                v14 = [x - r_pl, y, z_pl]
                v15 = [x - (math.cos(t) * r_pl), y + (math.sin(t) * r_pl), z_pl]
                v16 = [x + (math.cos(t) * r_pl), y + (math.sin(t) * r_pl), z_pl]
                v17 = [x + r_pl, y, z_pl]
                v18 = [x + (math.cos(t) * r_pl), y - (math.sin(t) * r_pl), z_pl]
                v19 = [x - (math.cos(t) * r_pl), y - (math.sin(t) * r_pl), z_pl]

                """Periferia"""
                v20 = [x - r_p, y, z_p]
                v21 = [x - (math.cos(t) * r_p), y + (math.sin(t) * r_p), z_p]
                v22 = [x + (math.cos(t) * r_p), y + (math.sin(t) * r_p), z_p]
                v23 = [x + r_p, y, z_p]
                v24 = [x + (math.cos(t) * r_p), y - (math.sin(t) * r_p), z_p]
                v25 = [x - (math.cos(t) * r_p), y - (math.sin(t) * r_p), z_p]
                
                # Exagerar copa ligeramente para que sea más visible
                z_ph = z_b + (z_ph - z_b) * 1.15  # eleva copa alta
                z_t  = z_b + (z_t - z_b) * 1.20   # sube la punta del árbol

                """Límite superior de la periferia"""
                v26 = [x - r_ph, y, z_ph]
                v27 = [x - (math.cos(t) * r_ph), y + (math.sin(t) * r_ph), z_ph]
                v28 = [x + (math.cos(t) * r_ph), y + (math.sin(t) * r_ph), z_ph]
                v29 = [x + r_ph, y, z_ph]
                v30 = [x + (math.cos(t) * r_ph), y - (math.sin(t) * r_ph), z_ph]
                v31 = [x - (math.cos(t) * r_ph), y - (math.sin(t) * r_ph), z_ph]

                """Copa del árbol"""
                v32 = [x, y, z_t]

                for i in range(32):
                    jsondict['vertices'].append(
                        list(np.around(eval("{}{}".format("v", i+1)), decimals=2))
                    )

                boundaries = []
                values = []

                """Crear índices para el modelo de árbol"""
                """Base del tronco del árbol"""
                for i in range(1, 7):
                    if i != 6:
                        boundaries.append([[vcounter, vcounter + i, vcounter + i + 1]])
                        values.append(0)
                    else:
                        boundaries.append([[vcounter, vcounter + i, vcounter + i - 5]])
                        values.append(0)
                """Tronco de árbol"""
                for i in range(1, 7):
                    if i != 6:
                        boundaries.append([[vcounter + i, vcounter + i + 6, vcounter + i + 7, vcounter + i + 1]])
                        values.append(0)
                    else:
                        boundaries.append([[vcounter + i, vcounter + i + 6, vcounter + i + 1, vcounter + i - 5]])
                        values.append(0)

                """Periferia inferior"""
                for i in range(7, 13):
                    if i != 12:
                        boundaries.append([[vcounter + i, vcounter + i + 6, vcounter + i + 7, vcounter + i + 1]])
                        values.append(colorvalue)
                    else:
                        boundaries.append([[vcounter + i, vcounter + i + 6, vcounter + i + 1, vcounter + i - 5]])
                        values.append(colorvalue)

                """Periferia"""
                for i in range(13, 19):
                    if i != 18:
                        boundaries.append([[vcounter + i, vcounter + i + 6, vcounter + i + 7, vcounter + i + 1]])
                        values.append(colorvalue)
                    else:
                        boundaries.append([[vcounter + i, vcounter + i + 6, vcounter + i + 1, vcounter + i - 5]])
                        values.append(colorvalue)

                """Periferia superior"""
                for i in range(19, 25):
                    if i != 24:
                        boundaries.append([[vcounter + i, vcounter + i + 6, vcounter + i + 7, vcounter + i + 1]])
                        values.append(colorvalue)
                    else:
                        boundaries.append([[vcounter + i, vcounter + i + 6, vcounter + i + 1, vcounter + i - 5]])
                        values.append(colorvalue)
                """Copa del árbol"""
                for i in range(25, 31):
                    if i != 30:
                        boundaries.append([[vcounter + 31, vcounter + i + 1, vcounter + i]])
                        values.append(colorvalue)
                    else:
                        boundaries.append([[vcounter + 31, vcounter + i - 5, vcounter + i]])
                        values.append(colorvalue)

                """
                jsondict['CityObjects'][tree[0]]['attributes'] = {
                    "TreeType": tree[23],
                    "Classification_Certainty": tree[24],
                    "Point Count": tree[3],
                    "Height Crown Base": z_c,
                    "Periphery Height": z_p,
                    "Periphery Radius": r_p,
                    "Lower Periphery Height": z_pl,
                    "Lower Periphery Radius": r_pl,
                    "Higher Periphery Height": z_ph,
                    "Higher Periphery Radius": r_ph,
                    "Tree Top": z_t,
                    "Ratio Height High + Low": tree[13],
                    "Ratio Height High + Periphery": tree[14],
                    "Ratio Height Periphery + Low": tree[15],
                    "Ratio Radius High + Low": tree[16],
                    "Ratio Radius High + Periphery": tree[17],
                    "Ratio Radius Periphery + Low": tree[18],
                    "Ratio Height Tree Top + Crown Base": tree[19],
                    "Average Intensity": tree[20],
                    "Average Number of Returns": tree[21],
                    "Ratio Periphery Height + Periphery Radius": tree[22]
                }
                """

                jsondict['CityObjects'][tree[0]]['attributes'] = {
                    "Segment ID": int(tree[0]),
                    "Point Count": int(tree[3]),
                    "Average Intensity": float(tree[20]),
                    "Average Number of Returns": float(tree[21])
                }

                jsondict['CityObjects'][tree[0]]['geometry'] = [{
                    "type": "MultiSurface",
                    "lod": lod,
                    "boundaries": boundaries,
                    "material": {
                        "visual": {
                            "values": values
                        },
                    },
                }]

                vcounter += 32

        with open("{}{}{}".format(path, "/Data/Output/", outfilename), 'w') as json_file:
            json.dump(jsondict, json_file, indent=2)

        print (time.perf_counter() - starttime)
        
    # Escritor para LODs 3.0 y 3.1
    if lod == 3:
        starttime = time.perf_counter()
        skipped_trees = 0
    
        max_trees = 50  # 👈 prueba con 20, 50, 100...
        max_points_per_tree = 50
        
        unique_ids = np.unique(tree_array['segment_id'])
    
        for idx_seg, seg_id in enumerate(unique_ids):
            if idx_seg >= max_trees:
                break

        for seg_id in np.unique(tree_array['segment_id']):
            rule = tree_array['segment_id'] == seg_id

            # puntos del árbol actual
            tree = tree_array[rule]
        
            if tree.shape[0] > max_points_per_tree:
                idx = np.random.choice(tree.shape[0], max_points_per_tree, replace=False)
                tree = tree[idx]

            # buscar parámetros de ese árbol en param_array
            index_param = np.where(np.array(param_array[:, 0], dtype=float) == seg_id)[0]
            if len(index_param) == 0:
                continue
                
            tree_param = np.array(param_array[index_param][0, :23], dtype=float)
            tree_type = "Unknown"
            type_certainty = float("nan")


            if len(tree) < 50:
                continue
             
            colorvalue = 1

            """ Genera materials.
            yellowtrees = ["Ailanthus", "Amelanchier", "Prunus"]
            mossytrees = ["Alnus", "Corylus", "Pyrus", "Robinia", "Styphnolobium"]
            blueishtrees = ["Fagus", "Catalpa"]
            vomittrees = ["Ginkgo", "Pinus"]
            grayishtrees = ["Liriodendron", "Malus", "Sorbus"]

            if tree_type in yellowtrees:
                colorvalue = 2
            if tree_type in mossytrees:
                colorvalue = 3
            if tree_type in blueishtrees:
                colorvalue = 4
            if tree_type in vomittrees:
                colorvalue = 5
            if tree_type in grayishtrees:
                colorvalue = 6
            """

            if tree_type == "Coniferae":
                colorvalue = 5
            x = tree['X'] / 1000.0
            y = tree['Y'] / 1000.0
            z = tree['height above ground'] / 1000.0  
            Z_real = tree['Z'] / 1000.0

            fit_3dfier = int(np.average(z - Z_real))
            z = z - fit_3dfier
            
            # Exagerar suavemente la copa en LOD3 (20% más alta)
            z_min = z.min()
            z = z_min + (z - z_min) * 1.20   # prueba con 1.15, 1.2, 1.3 según lo que veas
            
            pos = np.array((x, y, z)).T

            """Alpha shapes"""
            if not convex:
                convex = False
                alpha = 0.5

                xnorm = MinMaxScaler().fit_transform(x.reshape(-1, 1))
                ynorm = MinMaxScaler().fit_transform(y.reshape(-1, 1))
                znorm = MinMaxScaler().fit_transform(z.reshape(-1, 1))
                posnorm = np.array((xnorm, ynorm, znorm)).T[0]

                # Halla el radio de la circunsfera.
                # Por definición, el radio de la esfera que encaja dentro del tetraédrico debe ser menor que el valor alfa.

                tetrapos = np.take(posnorm, tetra.simplices, axis=0)
                normsq = np.sum(tetrapos ** 2, axis=2)[:, :, None]
                ones = np.ones((tetrapos.shape[0], tetrapos.shape[1], 1))
                a = np.linalg.det(np.concatenate((tetrapos, ones), axis=2))
                Dx = np.linalg.det(np.concatenate((normsq, tetrapos[:, :, [1, 2]], ones), axis=2))
                Dy = -np.linalg.det(np.concatenate((normsq, tetrapos[:, :, [0, 2]], ones), axis=2))
                Dz = np.linalg.det(np.concatenate((normsq, tetrapos[:, :, [0, 1]], ones), axis=2))
                c = np.linalg.det(np.concatenate((normsq, tetrapos), axis=2))

                r = np.sqrt((Dx ** 2 + Dy ** 2 + Dz ** 2) - (4 * a * c)) / (2 * np.abs(a))

                # Encuentra tetraédricos
                tetras = tetra.simplices[r < alpha, :]

                # triángulos
                TriComb = np.array([(0, 1, 2), (0, 1, 3), (0, 2, 3), (1, 2, 3)])
                Triangles = tetras[:, TriComb].reshape(-1, 3)
                Triangles = np.sort(Triangles, axis=1)

                # Eliminar los triángulos que aparecen dos veces, porque están dentro de las formas.
                TrianglesDict = defaultdict(int)
                for tri in Triangles:
                    TrianglesDict[tuple(tri)] += 1
                Triangles = np.array([tri for tri in TrianglesDict if TrianglesDict[tri] == 1])
                # bordes
                EdgeComb = np.array([(0, 1), (0, 2), (1, 2)])

                Edges = Triangles[:, EdgeComb].reshape(-1, 2)
                Edges = np.sort(Edges, axis=1)
                Edges = np.unique(Edges, axis=0)

                Vertices = np.unique(Edges)


                verts = np.hstack((np.where(Triangles[0][0] == Vertices)[0],
                                   np.where(Triangles[0][1] == Vertices)[0],
                                   np.where(Triangles[0][2] == Vertices)[0]))

                for triangle in Triangles[1:]:
                    verts = np.vstack((verts, np.hstack((np.where(triangle[0] == Vertices)[0],
                                                         np.where(triangle[1] == Vertices)[0],
                                                         np.where(triangle[2] == Vertices)[0]))))
                unsorted_triangles = verts
                vertices = pos[Vertices]
                boundaries = []
                values = []

            """Convex Hulls"""
            if convex:
                convexpoints = ConvexHull(pos).vertices
                convdelly = Delaunay(pos[convexpoints])
                unsorted_triangles = convdelly.convex_hull
                vertices = pos[convexpoints]
                boundaries = []
                values = []

            """Encuentra las medias aristas entre triángulos en una envoltura convexa y hace que solo las caras miren hacia afuera"""
            starttriangle = unsorted_triangles[0]
            neighborlist = []
            boundary_array = None

            sorted_triangles = sort_triangles(unsorted_triangles, starttriangle, neighborlist, boundary_array)
            if type(sorted_triangles) == bool:
                # Caso: Uno o más triángulos en la malla tienen más de 3 triángulos vecinos, esto no debería suceder
                skipped_trees += 1
                continue

            ccw_sorted_triangles = ccw_orientation(sorted_triangles, vertices)
            ccw_sorted_triangles = ccw_sorted_triangles + vcounter

            for boundary in ccw_sorted_triangles:
                boundaries.append([boundary.tolist()])
                values.append(colorvalue)

            vcounter += len(vertices)

            """Añadir tronco de árbol"""

              # Alturas reales del árbol
            z_min = Z_real.min()
            z_max = Z_real.max()
            height = z_max - z_min

            if height <= 0:
                # árbol raro, lo saltamos
                skipped_trees += 1
                continue

            # Centro en planta del árbol (XY promedio)
            x_center = float(x.mean())
            y_center = float(y.mean())

            # Radio "copa" aproximado como el radio máximo en XY
            dx = x - x_center
            dy = y - y_center
            radial_dist = np.sqrt(dx**2 + dy**2)
            r_crown = np.percentile(radial_dist, 90)  # radio de copa típico

            # Definimos el tronco:
            z_b = z_min                       # base del tronco (suelo)
            z_c = z_min + 0.55 * height        # tronco llega al 40% de la altura
            r_t = max(0.25 * r_crown, 0.15)   # radio del tronco = 25% del radio de copa, min 15 cm

            x = x_center
            y = y_center
            t = math.radians(60)

            """Centro, Suelo"""
            v1 = [x, y, z_b]

            """Tronco, Suelo"""
            v2 = [x - r_t, y, z_b]
            v3 = [x - (math.cos(t) * r_t), y + (math.sin(t) * r_t), z_b]
            v4 = [x + (math.cos(t) * r_t), y + (math.sin(t) * r_t), z_b]
            v5 = [x + r_t, y, z_b]
            v6 = [x + (math.cos(t) * r_t), y - (math.sin(t) * r_t), z_b]
            v7 = [x - (math.cos(t) * r_t), y - (math.sin(t) * r_t), z_b]

            """Parte superior del tronco"""
            v8 = [x - r_t, y, z_c]
            v9 = [x - (math.cos(t) * r_t), y + (math.sin(t) * r_t), z_c]
            v10 = [x + (math.cos(t) * r_t), y + (math.sin(t) * r_t), z_c]
            v11 = [x + r_t, y, z_c]
            v12 = [x + (math.cos(t) * r_t), y - (math.sin(t) * r_t), z_c]
            v13 = [x - (math.cos(t) * r_t), y - (math.sin(t) * r_t), z_c]

            """Centro, parte superior del tronco"""
            v14 = [x, y, z_c]

            """Límites del tronco del árbol"""
            boundaries2 = []
            values2 = []

            """Crear índices para el modelo de tronco de árbol implícito"""
            """Hexágono elevado en tierra"""
            for i in range(1, 7):
                if i != 6:
                    boundaries2.append([[vcounter, vcounter + i, vcounter + i + 1]])
                    values2.append(0)
                else:
                    boundaries2.append([[vcounter, vcounter + i, vcounter + i - 5]])
                    values2.append(0)

            """Lados hexagonales elevados"""
            for i in range(1, 7):
                if i != 6:
                    boundaries2.append([[vcounter + i, vcounter + i + 6, vcounter + i + 7, vcounter + i + 1]])
                    values2.append(0)
                else:
                    boundaries2.append([[vcounter + i, vcounter + i + 6, vcounter + i + 1, vcounter + i - 5]])
                    values2.append(0)

            """Copa de árbol hexagonal elevada"""
            for i in range(7, 13):
                if i != 12:
                    boundaries2.append([[vcounter + 13, vcounter + i + 1, vcounter + i]])
                    values2.append(0)
                else:
                    boundaries2.append([[vcounter + 13, vcounter + i - 5, vcounter + i]])
                    values2.append(0)

            vcounter += 14
            boundaries.extend(boundaries2)
            values.extend(values2)

            jsondict['CityObjects'][str(seg_id)] = {
                "type": "SolitaryVegetationObject",
            }

            jsondict['CityObjects'][str(seg_id)]['attributes'] = {
                "TreeType": tree_type,
                "Classification_Certainty": type_certainty,
                "Point Count": tree_param[3],
                "Average Intensity": np.around(tree_param[20], decimals=2),
                "Average Number of Returns": np.around(tree_param[21], decimals=2)
            }

            jsondict['CityObjects'][str(seg_id)]['geometry'] = [{
                "type": "MultiSurface",
                "lod": lod,
                "boundaries": boundaries,
                "material": {
                    "visual": {
                        "values": values
                    },
                },
            }]

            vertices = np.around(vertices, decimals=2).tolist()

            """Vertices for Alpha-Shapes or ConvexHulls"""
            for i in vertices:
                jsondict['vertices'].append(i)

            """Vertices para Troncos de árboles"""
            for i in range(14):
                jsondict['vertices'].append(
                    list(np.around(eval("{}{}".format("v", i + 1)), decimals=2))
                )
        # Eliminar atributos para reducir el peso del archivo
        for cid in jsondict['CityObjects']:
            jsondict['CityObjects'][cid].pop("attributes", None)

        # Redondear coordenadas a 2 decimales (reduce tamaño SIN dañar forma)
        jsondict['vertices'] = [
            [round(v[0], 2), round(v[1], 2), round(v[2], 2)] for v in jsondict['vertices']]
    
        # prettyprint
        with open("{}{}{}".format(path, "/Data/Output/", outfilename), 'w') as json_file:
            json.dump(jsondict, json_file, indent=2)

        # compact
        # with open("{}{}{}".format(path, "/Data/Output/FinalOutput/", outfilename), 'w') as json_file:
        #     json.dump(jsondict, json_file)

        print ("This many trees were not included due to a too low alpha value:", skipped_trees)
        print (time.perf_counter() - starttime)

def sort_triangles(unsorted_triangles, starttriangle, neighborlist, boundary_array, boundary_array_cond=False):
    if len(neighborlist) > 0:
        neighborlist = neighborlist[1:]

    i = unsorted_triangles[:, 0]
    j = unsorted_triangles[:, 1]
    k = unsorted_triangles[:, 2]

    l = starttriangle[0]
    m = starttriangle[1]
    n = starttriangle[2]

    neighbors = unsorted_triangles[np.logical_or(np.logical_or(np.logical_or(np.logical_or(np.logical_or(np.logical_or(
        np.logical_or(np.logical_or(np.logical_or(np.logical_or(np.logical_or(np.logical_or(
            np.logical_or(np.logical_or(np.logical_or(np.logical_or(np.logical_or(

                np.logical_and(np.logical_and(l == i, m == j), n != k),
                np.logical_and(np.logical_and(l == i, m != j), n == k)),
                np.logical_and(np.logical_and(l != i, m == j), n == k)),

                np.logical_and(np.logical_and(l == i, m == k), n != j)),
                np.logical_and(np.logical_and(l == i, m != k), n == j)),
                np.logical_and(np.logical_and(l != i, m == k), n == j)),

            np.logical_and(np.logical_and(l == j, m == k), n != i)),
            np.logical_and(np.logical_and(l == j, m != k), n == i)),
            np.logical_and(np.logical_and(l != j, m == k), n == i)),

            np.logical_and(np.logical_and(l == j, m == i), n != k)),
            np.logical_and(np.logical_and(l == j, m != i), n == k)),
            np.logical_and(np.logical_and(l != j, m == i), n == k)),

        np.logical_and(np.logical_and(l == k, m == i), n != j)),
        np.logical_and(np.logical_and(l == k, m != i), n == j)),
        np.logical_and(np.logical_and(l != k, m == i), n == j)),

        np.logical_and(np.logical_and(l == k, m == j), n != i)),
        np.logical_and(np.logical_and(l == k, m != j), n == i)),
        np.logical_and(np.logical_and(l != k, m == j), n == i)
    )]

    if boundary_array_cond == False:
        unsorted_triangles = np.delete(unsorted_triangles, np.where((starttriangle == unsorted_triangles).all(axis=1))[0], axis=0)
        boundary_array = np.array(starttriangle)
        boundary_array_cond = True

    if len(neighbors) > 3:
        return False
    for neighbor in neighbors:
        if np.ndim(boundary_array) == 2:
            ax_var = 1
        else:
            ax_var = None
        if not (starttriangle == boundary_array).all(axis=ax_var).any():
            boundary_array = np.vstack((boundary_array, starttriangle))
        if not (neighbor == boundary_array).all(axis=ax_var).any():
            if (np.flip(neighbor) == boundary_array).all(axis=ax_var).any():
                continue
            o, p, q = starttriangle[0], starttriangle[1], starttriangle[2]
            r, s, t = neighbor[0], neighbor[1], neighbor[2]
            if np.logical_or(np.logical_or(np.logical_or(np.logical_or(np.logical_or(np.logical_or(np.logical_or(np.logical_or(
                    np.logical_and(np.logical_and(o == r, p == s), q != t),
                    np.logical_and(np.logical_and(o == r, p != s), q == t)),
                    np.logical_and(np.logical_and(o != r, p == s), q == t)),

                    np.logical_and(np.logical_and(o == s, p == t), q != r)),
                    np.logical_and(np.logical_and(o == s, p != t), q == r)),
                    np.logical_and(np.logical_and(o != s, p == t), q == r)),

                    np.logical_and(np.logical_and(o == t, p == r), q != s)),
                    np.logical_and(np.logical_and(o == t, p != r), q == s)),
                    np.logical_and(np.logical_and(o != t, p == r), q == s)):
                boundary_array = np.vstack((boundary_array, np.flip(neighbor)))
                neighborlist.append(np.flip(neighbor))
            else:
                boundary_array = np.vstack((boundary_array, neighbor))
                neighborlist.append(neighbor)

    if len(neighborlist) > 0:
        next_starttriangle = neighborlist[0]
        return sort_triangles(unsorted_triangles, next_starttriangle, neighborlist, boundary_array, boundary_array_cond)
    else:
        sorted_triangles = boundary_array
        return sorted_triangles

def ccw_orientation(boundaries, vertices):
    triangles = vertices[boundaries]
    maxdex = np.argmax(np.average(triangles[:, :, 2], axis=1))
    bottom_triangle = triangles[maxdex]

    v0 = bottom_triangle[0]
    v1 = bottom_triangle[1]
    v2 = bottom_triangle[2]

    normal = np.cross(v1 - v0, v2 - v1)
    if normal[2] > 0:
        return boundaries
    else:
        return np.flip(boundaries)

Este bloque principal permite seleccionar el nivel de detalle (LOD 0–3) con el que se construirá el modelo geométrico de árboles en CityJSON. Dependiendo del LOD, se utilizan diferentes fuentes de datos: los modelos implícitos (LOD 0, 1 y 2) se generan a partir de parámetros geométricos como altura, radio de copa o intensidad media; mientras que el LOD 3 usa directamente los puntos LiDAR segmentados por árbol (segment_id) para construir una geometría realista mediante Convex Hull o Alpha-Shapes. Finalmente, se ejecuta la función write_cityJSON() para generar y guardar el modelo CityJSON resultante.

| LOD | Forma de árbol                               | Tipo de datos usados                   |
| --- | -------------------------------------------- | -------------------------------------- |
| 0   | Hexágono plano                               | Solo parámetros básicos                |
| 1   | Hexágono extruido (volumen simple)           | Altura y radio                         |
| 2   | Modelo paramétrico de copa y tronco          | Parámetros geométricos del árbol       |
| 3   | Geometría real (puntos LiDAR + Alpha Shapes) | Usa nube de puntos con `segment_id` |


In [23]:
if __name__ == '__main__':
    import os, sys

    path = r"D:\MODELO_3D\arboles"

    param_filename = False
    lod = 3       #LOD seleccionado
    convex = True

    if lod == 0:
        filename = "Noordereiland_segments_AllFeatures.npy"        # features por árbol
        param_filename = None
        convex = False
        outfilename = "Noordereiland_Classified_lod0.json"

    elif lod == 1:
        filename = "Noordereiland_segments_AllFeatures.npy"
        param_filename = None
        convex = False
        outfilename = "Noordereiland_Classified_lod1.json"

    elif lod == 2:
        filename = "Noordereiland_segments_AllFeatures.npy"
        param_filename = None
        convex = False
        outfilename = "Noordereiland_Classified_lod2.json"

    elif lod == 3:
        # 👇 aquí va el LOD 3:
        filename = "Noordereiland_segments_AllFeatures_full.npy"   # puntos con segment_id
        param_filename = "Noordereiland_segments_AllFeatures.npy"  # features por árbol
        convex = True # o False si quieres alpha-shape en vez de convex hull
        outfilename = "Noordereiland_Classified_lod3C.json"

    sys.setrecursionlimit(10000)
    write_cityJSON(path, filename, lod, outfilename, param_filename, convex)


This many trees were not included due to a too low alpha value: 0
7.71840259997407


## 6. Visualización

Para la visualización interactiva de los archivos generados en formato .cityjson, se utilizó la plataforma web CityJSON Ninja, disponible en:

🔗 https://ninja.cityjson.org/

CityJSON Ninja es un visor oficial desarrollado por la iniciativa CityJSON/CityGML, que permite cargar y explorar modelos 3D directamente en el navegador sin necesidad de instalar software adicional. Esta herramienta es particularmente útil para inspeccionar la correcta construcción de los objetos, revisar la integridad geométrica (vertices, boundaries, MultiSurface), verificar la orientación de las caras y validar el nivel de detalle (LOD) asignado a cada árbol.
